In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import localSettings as ls
import numpy as np
from load_data_run123 import *
from unblinding_far_sideband import *
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy.optimize import curve_fit
from plotting_tools import *
from ccncpi0_analysis import *

In [ ]:
ntuple_path = ls.ntuple_path+"../1102/unfiltered/"

In [ ]:
my_ccncpi0_analysis = ccncpi0_analysis(ntuple_path,addtlvars=[])

# Run the analysis to perform the closure test
## As presented at xsec mtg Jan 26,2021

In [ ]:
# Define as a function of which variable to perform the analysis.
# Options are: pi0enrg or pi0csth
var = 'pi0enrg'

v2d = var2dict[var]
bin_edges = v2d['bin_edges']

In [ ]:
effs, effsErrs = my_ccncpi0_analysis.efficiency_plot(var,ylim=(0.,0.5))

In [ ]:
effCC = effs['cc'][0]
effNC = effs['nc'][0]
print('effCC =',effCC)
print('effNC =',effNC)

In [ ]:
QUERY  = ACCEPTANCE+' and '+CCNCSEL

fCCsel, fCCselErr = my_ccncpi0_analysis.efficiency_plot(var,ylim=(0.,1.0),numqs=[CCSEL,NCSEL], denq=QUERY,\
                                                        pi0_types=['ccnc==0','ccnc==0'],colors=['k','g'],\
                                                        labels=['CCSEL','NCSEL'])
fNCsel, fNCselErr = my_ccncpi0_analysis.efficiency_plot(var,ylim=(0.,1.0),numqs=[CCSEL,NCSEL], denq=QUERY,\
                                                        pi0_types=['ccnc==1','ccnc==1'],colors=['k','g'],\
                                                        labels=['CCSEL','NCSEL'])

In [ ]:
fCC = fCCsel['CCSEL'][0]
fNC = fNCsel['NCSEL'][0]
print('fCC =',fCC)
print('fNC =',fNC)

In [ ]:
trueratio = my_ccncpi0_analysis.ccnc_plot_withratio(var,queries=['ccnc==0','ccnc==1'],\
                                                    colors=['b','r'],labels=['true CC','true NC'],\
                                                    title='truth, in FV',rylab='NC/CC')
print("trueratio =", trueratio)

In [ ]:
my_ccncpi0_analysis.ccnc_plot_withratio(var,acceptance=ACCEPTANCE+' and '+CCNCSEL,queries=['ccnc==0','ccnc==1'],\
                                        colors=['b','r'],labels=['true CC after CCNCSEL','true NC after CCNCSEL'],\
                                        title='truth, after selection',rylab='NC/CC')


In [ ]:
n,b,p = my_ccncpi0_analysis.ccnc_plot(var,acceptance=ACCEPTANCE,queries=[CCSEL,NCSEL],\
                                      colors=['b','r'],labels=['true pi0 after CCSEL','true pi0 after NCSEL'],\
                                      title='truth, after selection',reco=True)

recoCCsel = n[0]
recoNCsel = n[1]

print(np.sum(n,axis=1))
print('recoCCsel:',recoCCsel)
print('recoNCsel:',recoNCsel)

In [ ]:
my_ccncpi0_analysis.ccnc_resolution_plot('pi0enrg',ACCEPTANCE+' and '+CCNCSEL,\
                                         'truth, after selection','true pi0 after CCNCSEL',"Ereco-Etrue [GeV]",\
                                         [250., 0., 0.1])

In [ ]:
my_ccncpi0_analysis.ccnc_resolution_plot('pi0csth',ACCEPTANCE+' and '+CCNCSEL,\
                                         'truth, after selection','true pi0 after CCNCSEL',"CosThetaReco-CosThetaRtrue",\
                                         [250., 0., 0.1])

In [ ]:
ccsm = my_ccncpi0_analysis.smearing_matrix(var,ACCEPTANCE+' and '+CCSEL,'CCpi0 selection')
ncsm = my_ccncpi0_analysis.smearing_matrix(var,ACCEPTANCE+' and '+NCSEL,'NCpi0 selection')

In [ ]:
my_ccncpi0_analysis.closure_test(var,ccsm,recoCCsel,ncsm,recoNCsel,effCC,effNC,fCC,fNC,trueratio)

# Produce additional useful plots

In [ ]:
e,ee = my_ccncpi0_analysis.efficiency_plot(var,ylim=(0.,0.5),numqs=CCSEL, denq=ACCEPTANCE,\
                                           pi0_types=['ccnc==0','ccnc==1'],colors=['b','r'],\
                                           labels=['cc','nc'],ylab='CCSEL efficiency')
e,ee = my_ccncpi0_analysis.efficiency_plot(var,ylim=(0.,0.5),numqs=NCSEL, denq=ACCEPTANCE,\
                                           pi0_types=['ccnc==0','ccnc==1'],colors=['b','r'],\
                                           labels=['cc','nc'],ylab='NCSEL efficiency')


In [ ]:
my_ccncpi0_analysis.ccnc_plot_withratio(var,queries=[CCSEL+' and ccnc==0',NCSEL+' and ccnc==1'],\
                                        colors=['b','r'],labels=['true CC after CCSEL','true NC after NCSEL'],\
                                        title='truth, after selection',rylab='NC/CC')


In [ ]:
n,b,p = my_ccncpi0_analysis.ccnc_plot(var,acceptance=ACCEPTANCE,queries=[CCSEL,NCSEL],\
                                      colors=['b','r'],labels=['true pi0 after CCSEL','true pi0 after NCSEL'],\
                                      title='truth, after selection')
truetrueCCsel = n[0]
truetrueNCsel = n[1]

print(np.sum(n,axis=1))
print('truetrueCCsel:',truetrueCCsel)
print('truetrueNCsel:',truetrueNCsel)

In [ ]:
mySEL = ACCEPTANCE+' and '+CCNCSEL

plt.figure(figsize=(14, 4))
plt.subplot(121)
x = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0enrg']['varR']]
y = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0enrg']['varR']]-\
    my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0enrg']['varT']]
h = plt.hist2d(x, y, bins=[var2dict['pi0enrg']['bin_edges'],np.linspace(-1,1,16)])#, cmap=plt.cm.Greys)#, norm=LogNorm()
plt.xlabel(var2dict['pi0enrg']['labelR'])
plt.ylabel('Reco - '+var2dict['pi0enrg']['labelT'])
plt.colorbar()

plt.subplot(122)
x = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0csth']['varR']]
y = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0csth']['varR']]-\
    my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0csth']['varT']]
h = plt.hist2d(x, y, bins=[var2dict['pi0csth']['bin_edges'],np.linspace(-1,1,16)])#, cmap=plt.cm.Greys)#, norm=LogNorm()
plt.xlabel(var2dict['pi0csth']['labelR'])
plt.ylabel('Reco - '+var2dict['pi0csth']['labelT'])
plt.colorbar()
plt.show()

In [ ]:
mySEL = ACCEPTANCE+' and '+CCNCSEL

plt.figure(figsize=(14, 4))
plt.subplot(121)
x = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0mass']['varR']]
y = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0enrg']['varR']]-\
    my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0enrg']['varT']]
h = plt.hist2d(x, y, bins=[var2dict['pi0mass']['bin_edges'],np.linspace(-1,1,16)])#, cmap=plt.cm.Greys)#, norm=LogNorm()
plt.xlabel(var2dict['pi0mass']['labelR'])
plt.ylabel('Reco - '+var2dict['pi0enrg']['labelT'])
plt.colorbar()

plt.subplot(122)
x = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0mass']['varR']]
y = my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0csth']['varR']]-\
    my_ccncpi0_analysis.pi0s.query(mySEL)[var2dict['pi0csth']['varT']]
h = plt.hist2d(x, y, bins=[var2dict['pi0mass']['bin_edges'],np.linspace(-1,1,16)])#, cmap=plt.cm.Greys)#, norm=LogNorm()
plt.xlabel(var2dict['pi0mass']['labelR'])
plt.ylabel('Reco - '+var2dict['pi0csth']['labelT'])
plt.colorbar()
plt.show()

In [ ]:
my_ccncpi0_analysis.smearing_matrix('pi0enrg',ACCEPTANCE+' and '+CCNCSEL,'CCNCpi0 selection')
my_ccncpi0_analysis.smearing_matrix('pi0csth',ACCEPTANCE+' and '+CCNCSEL,'CCNCpi0 selection')

In [ ]:
#ccsm = a
#ncsm = a
#ccsm = b
#ncsm = b

mytrueCCsel = np.matmul(ccsm,recoCCsel)
print(trueCCsel)
mytrueNCsel = np.matmul(ncsm,recoNCsel)
print(trueNCsel)

#cheating
#trueCCsel = truetrueCCsel
#trueNCsel = truetrueNCsel

print('truetrueCCsel =',truetrueCCsel)
print('truetrueNCsel =',truetrueNCsel)
print('recoCCsel =',recoCCsel)
print('recoNCsel =',recoNCsel)
print('trueCCsel =',mytrueCCsel)
print('trueNCsel =',mytrueNCsel)
print('effCC =',effCC)
print('effNC =',effNC)
print('fCC =',fCC)
print('fNC =',fNC)
print('trueratio =',trueratio)